In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')
import datetime
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import statistics
np.random.seed(1)

In [2]:
import mxnet as mx
from mxnet import gluon
from gluonts.dataset.common import ListDataset
from gluonts.dataset.util import to_pandas
from gluonts.model import deepar
from gluonts.mx.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

In [3]:
import GPy, GPyOpt

### Setup

In [4]:
data = pd.read_csv("data.csv", index_col=0) # 1 + 1*4 + 1*4*7 + 1*4*7*2
agg_mat_df = pd.read_csv("agg_mat.csv", index_col=0) # matrix of aggregated data with bottom time series

In [5]:
data

,total,hol,vfr,bus,oth,nsw-hol,vic-hol,qld-hol,sa-hol,wa-hol,...,qld-oth-city,qld-oth-noncity,sa-oth-city,sa-oth-noncity,wa-oth-city,wa-oth-noncity,tas-oth-city,tas-oth-noncity,nt-oth-city,nt-oth-noncity
1998-03-31,84503,45906,26042,9815,2740,17589,10412,9078,3089,3449,...,431,271,244,73,168,37,76,24,35,8
1998-06-30,65312,29347,20676,11823,3466,11027,6025,6310,1935,2454,...,669,170,142,221,170,99,36,61,69,39
1998-09-30,72753,32492,20582,13565,6114,8910,5060,11733,1569,3398,...,270,1164,397,315,380,1166,32,23,150,338
1998-12-31,70880,31813,21613,11478,5976,10658,5481,8109,2270,3561,...,214,535,194,260,410,1139,48,43,172,453
1999-03-31,86893,46793,26947,10027,3126,16152,10958,10047,3023,4287,...,458,557,147,33,162,28,77,60,15,47
1999-06-30,66866,31442,19481,11875,4068,9840,5841,8088,2219,3905,...,364,555,342,129,125,161,70,90,72,63
1999-09-30,72182,34623,20026,11713,5820,9265,5104,12553,1712,3729,...,935,1865,137,109,546,243,26,49,75,182
1999-12-31,68318,31118,20431,10582,6187,10427,4774,9652,1940,2732,...,640,703,400,251,299,568,34,56,145,217
2000-03-31,85651,47030,24284,9734,4603,16340,9972,10077,3282,5131,...,513,374,274,155,365,266,49,122,37,56
2000-06-30,64467,30852,19430,10590,3595,9797,5758,8564,1845,3324,...,298,249,348,84,212,299,33,47,41,45


In [6]:
# Split the data
### pivot data such as index is the name of columns
#data = data.pivot(index='date', columns='symbol', values='close')
pivot_df = data.T

#X_train = pivot_df.iloc[:,:20]
#X_val = data.iloc[nb_train:nb_train+nb_val,:]
#X_test = pivot_df.iloc[:,8:28]

#y_train = pivot_df.iloc[:,20:28]
#y_val = data.iloc[nb_train+nb_val,:]
y_test = pivot_df.iloc[:,28:]

input_size = 15
prediction_length = 8
pred_length = prediction_length
start_date = '1998-03-31'
freq = "Q"

In [7]:
level0ag = 1
level0total = 1
level1ag = 4
level1total = level0total*level1ag
level2ag = 7
level2total = level1total*level2ag
level3ag = 2
level3total = level2total*level3ag

In [8]:

levels = [0, level0ag,level0ag*level1ag,level0ag*level1ag*level2ag,level0ag*level1ag*level2ag*level3ag]
levels_left = [0, level0total, level0total+level1total, level0total+level1total+level2total]
levels_right = [0, level1total, level1total+level2total, level1total+level2total+level3total]
nb_ts_levels = [level0total, level1total, level2total, level3total]
nb_ts_agg_levels = [level0ag,level1ag,level2ag,level3ag]
lengths = nb_ts_levels
total_ts = [0,level0total, level0total+level1total, level0total+level1total+level2total, level0total+level1total+level2total+level3total]

In [9]:
def calculate_wmape(actual_values, forecasted_values):
    n = len(actual_values)
    num = np.sum(np.abs(actual_values - forecasted_values))
    den = np.sum(np.abs(actual_values))
    wmape = 100*num/den
    return wmape

In [10]:
data_for_model = data

## DeepAR

In [11]:
def deepAR_ds(data_for_model, prediction_length, freq, start):
    # train dataset: cut the last window of length "prediction_length", add "target" and "start" fields
    train_ds = ListDataset([{'target': data_for_model[x][:-prediction_length], 'start': start}
                        #'feat_static_cat':feat_static_cat[x].values}
                        #'feat_dynamic_cat':[feat_dynamic_cat_month[x][:-prediction_length]]}
                        for x in data_for_model.columns],
                        freq=freq)
    # test dataset: use the whole dataset, add "target" and "start" fields
    test_ds = ListDataset([{'target': data_for_model[x].values, 'start': start}
                        #'feat_static_cat':feat_static_cat[x].values}
                        #'feat_dynamic_cat':[feat_dynamic_cat_month[x].values]}
                        for x in data_for_model.columns],
                        freq=freq)
    return train_ds, test_ds

In [12]:
def deepAR_fit(train_ds, test_ds, prediction_length, freq, learning_rate, cell_type, num_layers, num_cells, num_epochs, num_batches_per_epoch):
    
    trainer = Trainer(epochs=num_epochs, learning_rate=learning_rate,num_batches_per_epoch=num_batches_per_epoch)
    
    estimator = deepar.DeepAREstimator(
        freq=freq, prediction_length=prediction_length, trainer=trainer, cell_type=cell_type,
        num_layers=num_layers,num_cells=num_cells)
    
    predictor = estimator.train(training_data=train_ds)
    
    return predictor

In [13]:
def deepAR_predict(predictor, test_ds):

    forecast_it, ts_it = make_evaluation_predictions(
        dataset=test_ds,  # test dataset
        predictor=predictor,  # predictor
        num_samples=100,  # number of sample paths we want for evaluation
    )
    forecasts = list(forecast_it)
    tss = list(ts_it)
    
    return forecasts,tss

In [14]:
def run_deepAR(data_for_model, prediction_length, freq, start, learning_rate, cell_type, num_layers, num_cells, num_epochs, num_batches_per_epoch):
    train_ds, test_ds = deepAR_ds(data_for_model, prediction_length, freq, start)
    predictor = deepAR_fit(train_ds, test_ds, prediction_length, freq, learning_rate, cell_type, num_layers, num_cells, num_epochs, num_batches_per_epoch)
    forecasts,tss = deepAR_predict(predictor, test_ds)
    return forecasts,tss

#### b) Fine-Tuning

In [15]:
def run_deepAR_FT(data_for_model, prediction_length, freq, start, learning_rate, cell_type, num_layers, num_cells, num_epochs, num_batches_per_epoch):
    train_ds, test_ds = deepAR_ds(data_for_model, prediction_length, freq, start)
    predictor = deepAR_fit(train_ds, test_ds, prediction_length, freq, learning_rate, cell_type, num_layers, num_cells, num_epochs, num_batches_per_epoch)
    forecasts,tss = deepAR_predict(predictor, test_ds)
    evaluator = Evaluator()
    agg_metrics, item_metrics = evaluator(tss, forecasts)
    return agg_metrics['RMSE']

In [19]:
def optimize_on_metric(data_for_model, prediction_length, freq, start):
    # bounds for hyper-parameters
    # the bounds dict should be in order of continuous type and then discrete type
    bounds = [{'name': 'learning_rate', 'type': 'discrete',  'domain': (0.001, 0.05, 0.01)},
              {'name': 'num_layers', 'type': 'discrete',    'domain': (3, 4, 5)},
              {'name': 'num_cells', 'type': 'discrete',    'domain': (50,60)},
              {'name': 'num_batches_per_epoch', 'type': 'discrete',    'domain': (5, 10, 32)},
              {'name': 'epochs', 'type': 'discrete', 'domain': (10, 50, 100, 150)}
              #{'name': 'cell_type', 'type': 'discrete', 'domain': ("lstm", "gru")}               
              ]

    def f(x):
        print(x)
        evaluation = run_deepAR_FT(
            data_for_model, prediction_length, freq, start,
            learning_rate = float(x[:,0]),  
            cell_type= "lstm",
            num_layers= int(x[:,1]), 
            num_cells= int(x[:,2]),
            num_batches_per_epoch= int(x[:,3]), 
            num_epochs=  int(x[:,4]))
        print("LOSS:\t{0}".format(evaluation))
        print(evaluation)
        return evaluation

    opt_transformer = GPyOpt.methods.BayesianOptimization(f=f, domain=bounds)

    opt_transformer.run_optimization(max_iter=10)

    print("RESULTS:")
    print(opt_transformer.x_opt) 
    print(opt_transformer.fx_opt)

    return opt_transformer

In [20]:
opt_sol = optimize_on_metric(data_for_model, prediction_length, freq, start_date)

[[1.e-03 4.e+00 6.e+01 1.e+01 1.e+01]]


100%|██████████| 10/10 [00:00<00:00, 19.13it/s, epoch=10/10, avg_epoch_loss=7.44]
Running evaluation: 89it [00:00, 181.45it/s]


LOSS:	1249.660653872944
1249.660653872944
[[5.0e-02 4.0e+00 6.0e+01 3.2e+01 5.0e+01]]


100%|██████████| 32/32 [00:03<00:00,  9.80it/s, epoch=50/50, avg_epoch_loss=7.36]
Running evaluation: 89it [00:00, 126.95it/s]


LOSS:	907.2551228836478
907.2551228836478
[[1.e-02 5.e+00 6.e+01 1.e+01 1.e+01]]


...

LOSS:	1447.6961896917153
1447.6961896917153
[[5.0e-02 4.0e+00 6.0e+01 3.2e+01 5.0e+01]]


100%|██████████| 32/32 [00:03<00:00,  8.46it/s, epoch=50/50, avg_epoch_loss=7.33]
Running evaluation: 89it [00:01, 76.11it/s]


LOSS:	1268.5501676868473
1268.5501676868473
[[5.0e-02 4.0e+00 6.0e+01 3.2e+01 5.0e+01]]


100%|██████████| 32/32 [00:02<00:00, 14.79it/s, epoch=50/50, avg_epoch_loss=6.98]
Running evaluation: 89it [00:00, 192.29it/s]

LOSS:	727.4693941612077
727.4693941612077
RESULTS:
[5.0e-02 4.0e+00 6.0e+01 3.2e+01 5.0e+01]
727.4693941612077


#### c) Final Prediction

In [35]:
### Prediction for first class
forecasts0,tss0 = run_deepAR(data, prediction_length, freq, start_date, 0.05, "lstm", 4, 60, 100, 32)

  0%|          | 0/32 [00:00<?, ?it/s]

100%|██████████| 32/32 [00:02<00:00, 11.32it/s, epoch=100/100, avg_epoch_loss=7]


In [36]:
### round to int value of array
def round_array(array):
    for i in range(len(array)):
        array[i] = round(array[i])
        if array[i] <= 0:
            array[i] = 0
    return array

In [37]:
### create dataframe with predictions
def create_df_deepar(forecast, data_for_model):
    ### dataframe with name of columns same as in data_for_model_000
    df = pd.DataFrame(columns=data_for_model.columns)
    for i,col in enumerate(data_for_model.columns):
        df[col] = round_array(forecast[i].median)
    return df

In [38]:
y_predict = create_df_deepar(forecasts0, data)
y_predict = y_predict.T
y_predict.columns = y_test.columns

In [39]:
y_predict

,2005-03-31,2005-06-30,2005-09-30,2005-12-31,2006-03-31,2006-06-30,2006-09-30,2006-12-31
total,81063.0,65320.0,71069.0,72217.0,78919.0,64600.0,70750.0,72183.0
hol,40078.0,29424.0,32208.0,31558.0,39929.0,28889.0,31871.0,31296.0
vfr,28156.0,22994.0,23085.0,24415.0,27610.0,23008.0,22885.0,24241.0
bus,9404.0,9914.0,11462.0,11216.0,9650.0,9933.0,11148.0,10965.0
oth,3600.0,3696.0,5272.0,5918.0,3294.0,3751.0,4987.0,5507.0
...,...,...,...,...,...,...,...,...
wa-oth-noncity,121.0,171.0,409.0,649.0,104.0,163.0,428.0,622.0
tas-oth-city,103.0,64.0,85.0,57.0,88.0,62.0,72.0,44.0
tas-oth-noncity,73.0,100.0,54.0,48.0,51.0,98.0,68.0,37.0
nt-oth-city,3.0,34.0,107.0,128.0,12.0,40.0,141.0,97.0


In [40]:
y_test

,2005-03-31,2005-06-30,2005-09-30,2005-12-31,2006-03-31,2006-06-30,2006-09-30,2006-12-31
total,85992,59637,66846,63392,82637,67523,65938,69544
hol,44043,26719,30947,26418,43601,30777,30938,31845
vfr,28811,19635,19743,19954,26245,22948,19751,22758
bus,9084,9161,11000,11547,8712,10099,11352,10784
oth,4054,4122,5156,5473,4079,3699,3897,4157
...,...,...,...,...,...,...,...,...
wa-oth-noncity,105,250,421,385,139,234,213,419
tas-oth-city,76,31,37,50,36,84,33,67
tas-oth-noncity,96,49,17,53,236,85,68,37
nt-oth-city,86,91,196,49,50,29,77,27


#### WMAPE

In [41]:
def wmape_level(actual_value, forecasted_value, total_ts, lengths):
    nb_levels = len(lengths)
    wmapes = []
    for l in range(nb_levels):
        actual_value_ts = actual_value[total_ts[l]:total_ts[l+1], :]
        forecasted_value_ts = forecasted_value[total_ts[l]:total_ts[l+1], :]
        wmapes.append(calculate_wmape(actual_value_ts, forecasted_value_ts))
    return wmapes

In [42]:
wmape_level(y_test.to_numpy(), y_predict.to_numpy(), total_ts, lengths)

[6.723311647720696, 8.200224751517785, 12.286713124811891, 14.975004852994342]

In [43]:
calculate_wmape(y_test.to_numpy(), y_predict.to_numpy())

10.546313594261179

### RMSSE

In [44]:
### I have an array of shape (89,5)
### create dataframe with predictions
def create_df(y_predict, pred_length, data):
    ### dataframe with name of columns same as in data_for_model_000
    ### create a dataframe based on data, remove last pred_length rows, and add y_predict
    ### return dataframe
    y_predict_df = y_predict.astype(np.float32)
    y_predict_df = pd.DataFrame(y_predict_df)
    y_predict_df = y_predict_df.T
    df = data.copy()
    for i,col in enumerate(data.columns):
        df[col][-(pred_length):] = y_predict_df[:][i]
    return df

In [45]:
data_pred = create_df(y_predict.to_numpy(), pred_length, data)

In [46]:
def rmsse_ts(pred_length, data, data_pred, ts):
    H = pred_length
    T = data.shape[0] - H
    ts_array = data.iloc[:,ts].values
    ts_array_pred = data_pred.iloc[:,ts].values
    e = (1/H)*np.sum((ts_array[t] - ts_array_pred[t])**2 for t in range(T, T+H))
    e_naive = (1/(T-1))*np.sum((ts_array[t] - ts_array[t-1])**2 for t in range(1, T))
    return np.sqrt(e/e_naive)

In [47]:
total_ts = [0,1,5,5+28,5+28+56]
lengths = [1, 4, 28, 56]
def rmsse_level(pred_length, data, data_pred, total_ts, lengths):
    nb_levels = len(lengths)
    R = 0
    r_l = [0]*nb_levels
    for l in range(nb_levels):
        for j in range(total_ts[l], total_ts[l+1]):
            r_l[l] += (1/lengths[l])*rmsse_ts(pred_length, data, data_pred, j)
            #print(l, j)
    print(r_l)
    R += np.mean(r_l)
    return R

In [48]:
rmsse_level(pred_length, data, data_pred, total_ts, lengths)

[0.4246614634631024, 0.44223607962482014, 0.5440185029923672, 0.5924670302676679]


0.5008457690869894